## Schedule data
Schedule the input spikes to the SNN appropriately for a 3x3 Kernal sweep


*   Padding = 0
*   Stride = 1
*   Kernal = 3x3



In [ ]:
import os
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Kernel wise sweep of data

In [ ]:
# Read the multi-channel data from the input file
def read_input_data(file_path):
    channels = []
    current_channel = []
    with open(file_path, 'r') as f:
        for line in f:
            stripped_line = line.strip()
            if stripped_line.startswith("Sparsity:"):
                continue  # Skip the sparsity line
            if stripped_line.startswith("Channel"):
                if current_channel:
                    channels.append(np.array(current_channel))
                    current_channel = []
            elif stripped_line:
                row = list(map(float, stripped_line.split()))
                current_channel.append(row)
        if current_channel:
            channels.append(np.array(current_channel))
    return channels

# Add padding to the data to make it 16x16
def pad_data(data, pad_width=1):
    return np.pad(data, pad_width=pad_width, mode='constant', constant_values=0)

# Prepare the data scheduling
def schedule_data(padded_data, kernel_size=3, stride=1):
    rows, cols = padded_data.shape
    schedule = []

    for i in range(0, rows - kernel_size + 1, stride):
        for j in range(0, cols - kernel_size + 1, stride):
            window = padded_data[i:i + kernel_size, j:j + kernel_size].flatten()
            schedule.append(window)

    return np.array(schedule, dtype=int)  # Convert to integers

def write_scheduled_data(file_path, scheduled_data):
    with open(file_path, 'w') as f:
        for channel_idx, channel_data in enumerate(scheduled_data):
            f.write(f'Channel {channel_idx}:\n')
            for row in channel_data:
                f.write(' '.join(map(str, row)) + '\n')

# Main function to execute the steps
def main(input_file_path, output_file_path):
    channels = read_input_data(input_file_path)
    scheduled_channels = []

    for channel in channels:
        # padded_data = pad_data(channel)
        scheduled_data = schedule_data(channel)
        scheduled_channels.append(scheduled_data)

    write_scheduled_data(output_file_path, scheduled_channels)
    print(f"Scheduled data has been written to {output_file_path}")


Scheduled data has been written to /content/drive/My Drive/project_aug/v4/input_output_v5/inspk_8b_schedld0.txt


In [ ]:
base_path = '/content/drive/My Drive/project_aug/v4'

timesteps = 3

# call the main function
for timestep in range(timesteps):
  input_file_path = base_path + f'/input_output_v5/in_spk_8b{timestep}.txt'
  output_file_path = base_path + f'/input_output_v5/inspk_8b_schedld{timestep}.txt'
  main(input_file_path, output_file_path)

In [ ]:
# # Read the uploaded file
# input_filename = base_path + '/input_output_v5/inspk_8b_schedld0.txt'

# def read_file(filename):
#     with open(filename, 'r') as file:
#         data = file.readlines()
#     return data

# data = read_file(input_filename)

# # Parse the data into a dictionary of channels
# def parse_data(data):
#     channels = {}
#     current_channel = None

#     for line in data:
#         if line.startswith("Channel"):
#             current_channel = int(line.split()[1].strip(':'))
#             channels[current_channel] = []
#         elif current_channel is not None:
#             spikes = list(map(int, line.split()))
#             channels[current_channel].append(spikes)

#     return channels

# channels = parse_data(data)

# # Transform the data into the desired format
# def transform_data(channels):
#     transformed = []
#     num_channels = len(channels)
#     num_cycles = len(channels[0])

#     for i in range(num_cycles):
#         for j in range(len(channels[0][i])):
#             row = []
#             for ch in range(num_channels):
#                 row.append(channels[ch][i][j])
#             transformed.append(row)

#     return transformed

# transformed_data = transform_data(channels)
# # Write the transformed data to a new file
# output_filename =  base_path + '/input_output_v5/inspk_8b_vhdl0.txt'

# def write_file(filename, data):
#     with open(filename, 'w') as file:
#         for row in data:
#             file.write(''.join(map(str, row)) + '\n')

# write_file(output_filename, transformed_data)